# Article figures

In [1]:
import numpy as np
import scipy.linalg as la
import cvxpy as cvx
import pandas as pd

import os
import pickle
import plotly.express as px

import hurwitz
import cvx_inference

Starting up Julia...


┌ Warning: Replacing docs for `SciMLOperators.AbstractSciMLOperator :: Union{}` in module `SciMLOperators`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `SciMLBase.sol :: Union{Tuple, Tuple{D}, Tuple{S}, Tuple{N}, Tuple{T}} where {T, N, S, D}` in module `SciMLBase`
└ @ Base.Docs docs/Docs.jl:240


## Introduction

## Methods

### [X] SCS reduction diagram

In [2]:
n = 5
A = np.array(
    [
        [-2.,0,1.,0,1.],
        [0,-2.,0,1.,0],
        [0,0,-2.,0,1.],
        [-1,0,0,-2.,0],
        [0,-1,0,0,-2.]
     ])
gamma = hurwitz.analytic_gamma(A)
E = np.abs(np.sign(A))

u, U = la.eig(gamma)

Alpha_mat = np.full(((n*(n+1))//2,n**2), 0.0)
beta_vec = np.full(((n*(n+1))//2), 0.0)
zero_inds = np.full((n**2), 0)

c1 = 0
for i in range(n):
    for j in range(i,n):

        if i!=j:
            beta_vec[c1] = 0
        else:
            beta_vec[c1] = -1/(2*u[i]).real
        c2 = 0
        for l in range(n):
            for k in range(n):
                if E[l,k] == 0: zero_inds[c2] = 1
                if (i!=j):
                    Alpha_mat[c1, c2] = cvx_inference.alphas(i, j, l, k, U, u).real
                else:
                    Alpha_mat[c1, c2] = cvx_inference.betas(i, l, k, U).real
                c2 += 1
        c1 += 1

vx = cvx.Variable(n**2)
objective = cvx.Minimize(cvx.norm(cvx.multiply(zero_inds,vx), 1))
constraints = [Alpha_mat@vx == beta_vec]
prob = cvx.Problem(objective, constraints)
probdata, chain, inverse_data = prob.get_problem_data(cvx.SCS)
bbb = probdata["b"].reshape((len(probdata["b"]), 1))
fig = px.imshow(
    probdata["A"].toarray(), 
    color_continuous_midpoint=0.0, 
    color_continuous_scale=px.colors.sequential.RdBu
    )
fig.update_traces(dict(
    
))
fig.update_xaxes(dict(
    showticklabels = False
))
fig.update_yaxes(dict(
    showticklabels = False
))
fig.show()

In [5]:
fig.write_image("figs/SCSmatrix.pdf")

### [X] Example images of reconstruction methods on 10x10

## Results

In [6]:
import experiments
n = 10
experiments.gen_rand_timeseries(Ns = [n], edge_probs=[n*0.1], mats_num=100)

100%|██████████| 100/100 [18:35<00:00, 11.16s/it]


### [X] Violins of accuracy in different methods

In [7]:
# import experiments
# n = 10
# experiments.run_E_inference(Ns = [n], edge_probs=[n*0.1], mats_num=100, alphas = [0.01, 0.05, 0.1])
# experiments.run_A_inference(Ns = [n], edge_probs=[n*0.1], mats_num=100, alphas = [0.01, 0.05, 0.1])

100%|██████████| 100/100 [8:59:32<00:00, 323.72s/it]  


In [25]:
N = 10
step_size = 0.1
edge_prob = N*0.1
pkl_obj = pickle.load(open("pickles/results/inferences/N{}_ss{:.2f}_ep{:.2f}_as.pkl".format(N, step_size, edge_prob), "rb"))

err_df_lst = []

for mat_num in pkl_obj.keys():
    A = pkl_obj[mat_num]["A"]
    for alpha in pkl_obj[mat_num].keys():
        if not type(alpha) is float: continue
        # for method in pkl_obj[mat_num][alpha]["A Inference"].keys():
        for method in ["Full Info + LP", "TE + LP", "No Info + LP"]:
            for rep in range(len(pkl_obj[mat_num][alpha]["A Inference"][method])):
                A_inf = pkl_obj[mat_num][alpha]["A Inference"][method][rep]
                err_df_lst.append([N, mat_num, alpha, rep, method, hurwitz.calc_mat_dist(A,A_inf)])
                
err_df = pd.DataFrame(err_df_lst, columns=["Size", "Mat", "Alpha", "Rep", "Method", "Error"])

In [30]:
plt = px.violin(err_df[(err_df["Alpha"] == 0.01) & (err_df["Method"].isin(["Full Info + LP", "TE + LP", "No Info + LP"]))], x = "Method", y = "Error", color="Method", box=True, range_y=[0,0.25], points="outliers")
plt.show()

In [ ]:
plt.write_image("figs/SCSmatrix.svg")

### [ ] Plot of accuracy/alignment vs. edge probability

### [ ] An example with external noise ? 

### [ ] Examples with non-linear (saturating) systems
Imagino que para este caso podría ser una figura que muestre ejemplos particulares de la matriz de conectividad, la reconstrucción, y los valores de alineación 